# Homework 3 - What is the best anime in the world?

## 1. Data collection

In [7]:
import requests as r
from bs4 import BeautifulSoup
from tqdm import tqdm
import pandas as pd
pd.options.mode.chained_assignment = None  # default='warn'
import os
from datetime import datetime
import csv
import re
import nltk
from collections import Counter
import numpy as np
import math
import heapq
import time
import numpy as np
import re
from nltk.corpus import wordnet
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer # or LancasterStemmer, RegexpStemmer, SnowballStemmer
import string
import json

default_stemmer = PorterStemmer()
default_stopwords = stopwords.words('english')

# Append some recurrent words in synopsis:
frequent_words = ['character','characters','end']
default_stopwords.extend(frequent_words)

#### List of paths

In [8]:
path_list_anime = "C:/Users/andre/Desktop/Anime/list_anime.txt"
url_of_single_page = "https://myanimelist.net/topanime.php?limit="
path_page_general = "C:/Users/andre/Desktop/Anime/Page"
path_documents_file = "C:/Users/andre/Desktop/Anime/documents.tsv"
path_vocabulary_file = "C:/Users/andre/Desktop/Anime/vocabulary.json"
path_index_file = "C:/Users/andre/Desktop/Anime/index.json"
path_documents_ex_3 = "C:/Users/andre/Desktop/Anime/documents_ex_3.tsv"
path_vocabulary_title_file = "C:/Users/andre/Desktop/Anime/vocabulary_title.json"
path_index_title_file ="C:/Users/andre/Desktop/Anime/index_title.json"

### 1.1. Get the list of animes

#### Write a file.txt in which each row is a anime url

In [244]:
# 19130 anime in total, 383 pages with 50 animes

def write_list_anime(path_list_anime, url_of_single_page):
    
    # Number of total pages
    n_pages = 383 

    with open(path_list_anime, "w", encoding='utf-8') as file:

        for page in tqdm(range(0, n_pages)):

            # URL of the single page 
            url = url_of_single_page + str(page * 50)

            # Get response object
            response = r.get(url)

            # Get html
            soup = BeautifulSoup(response.text, 'html.parser')

            # Find all links iterating through the document
            for tag in soup.find_all("tr", class_="ranking-list"):
                a_list = tag.find_all('a', class_="hoverinfo_trigger fl-l ml12 mr8" ,href=True)
                for a in a_list:
                    link = a['href']
                    file.write(str(link) + '\n')

    file.close()

### 1.2. Crawl animes

#### Dowload the html of each url in file.txt

In [247]:
def download_html(path_list_anime, path_page_general):
    file = open(path_list_anime, "r", encoding='utf-8')

    # Initialize counters 
    counter_anime = 0
    page = 0

    # Get html file for every anime (url is a line in the file list_anime) 
    for line in tqdm(file):
        # Every 50 animes the program stops because of the scraping block of the website
        time.sleep(1.5)
        counter_anime += 1
        if (counter_anime%50 == 1):
            page +=1        
        response = r.get(line)
        soup = BeautifulSoup(response.text, 'html.parser')

        try:
            os.mkdir(path_page_general + str(page))
        except OSError:
            pass

        with open(path_page_general + str(page) + "/article_" + str(counter_anime) + ".html", "w", encoding='utf-8') as file:
            file.write(str(soup))

In [248]:
#download_html(path_list_anime, path_page_general)

202it [08:41,  2.58s/it]


KeyboardInterrupt: 

### 1.3 Parse downloaded pages

#### Function to retrieve all the information from html

In [263]:
# Retrive information at page i of anime j
def retrive_information(i, j):
        with open(path_page_general + str(i) + "/article_" + str(j) + ".html", "r", encoding='utf-8') as file:
    
            # Read file and parse html code
            content_page = file.read()
            soup = BeautifulSoup(content_page, 'html.parser')
            
            # Retrive information
            
            animeTitle = soup.find_all("h1", class_="title-name h1_bold_none")[0].find("strong").text
        
            animeType = soup.find_all('a', href = re.compile(r'type*'))[0].text.strip()
            
            animeNumEpisode = soup.find(string='Episodes:').next_element.replace("\n","").strip()
            
            animeNumMembers = soup.find(string='Members:').next_element.replace("\n","").replace(',','').replace('#', '').strip()
        
            animeScore = soup.find('span', {"itemprop":"ratingValue"}).text.strip()
            
            animeUsers = soup.find('span', {"itemprop":"ratingCount"}).text.strip()
            
            animeRank = soup.find(string='Ranked:').next_element.replace("\n","").replace('#','').strip()
            
            animePopularity = soup.find(string='Popularity:').next_element.replace("\n","").replace('#', '').strip()
            
            animeDescription = soup.find('p',{'itemprop':'description'}).text.replace('\n', '').replace('  ', '').strip()

    
            # Iterate in di with dates   
            dates_div = soup.find_all("div", class_="spaceit_pad")
            for i in range(0, len(dates_div)):
        
                # Dates
                if dates_div[i].find("span", string='Aired:') != None:
                    dates = dates_div[i].contents[2].strip().split("to")
                    try:
                        releaseDate = datetime.strptime(dates[0].strip(), '%b %d, %Y').date()
                    except:
                        releaseDate = None
                    try:
                        endDate = datetime.strptime(dates[1].strip(), '%b %d, %Y').date()
                    except:  
                        endDate = None
    
            # Anime Description
            animeDescription = soup.find("p", itemprop='description').text
    
            # Related Animes
            try:
                related_anime_table = soup.find("table", class_="anime_detail_related_anime")
                links_list = [link.text for link in related_anime_table.find_all("a")]
                animeRelated = list(set(links_list))
            except:
                animeRelated = None
    
            # Characters
            animeCharacters = soup.find_all("h3", class_="h3_characters_voice_actors")
            animeCharacters = [character.text for character in animeCharacters]
    
            # Voices
            animeVoices = soup.find_all("td", class_="va-t ar pl4 pr4")
            animeVoices = [actor.find("a").text for actor in animeVoices]
    
            a_no = []
            total = []
            result = []
            animeStaff = []
    
            # Staff 
            if animeCharacters == [] and animeVoices == []:
                div = soup.find_all("div", class_="detail-characters-list clearfix")[0]
                tds = div.find_all("td", class_=lambda x: x != 'ac borderClass' and x != 'ac' and x == 'borderClass')
    
                smalls = div.find_all("small")
    
                for td in tds:
                    a_no = set(list(td.find_all("a", class_="fw-n")))
                    total = set(list(td.find_all("a")))
                    if list(total.difference(a_no)) != []:
                        result.append(list(total.difference(a_no)))
    
                for (s, role) in zip(result, smalls):
                    s = str(s).strip("[").strip("]")
                    htmlTag = BeautifulSoup(s, "html.parser")
                    animeStaff.append([htmlTag.text, role.text])
                
            # There are no staff
            elif len(soup.find_all("div", class_="detail-characters-list clearfix")) < 2:
                animeStaff = []
                
            # There are one of animeCharacters or animeVoices
            else:
                div = soup.find_all("div", class_="detail-characters-list clearfix")[1]
                
                tds = div.find_all("td", class_=lambda x: x != 'ac borderClass' and x != 'ac' and x == 'borderClass')

                smalls = div.find_all("small")

                for td in tds:
                    a_no = set(list(td.find_all("a", class_="fw-n")))
                    total = set(list(td.find_all("a")))
                    if list(total.difference(a_no)) != []:
                        result.append(list(total.difference(a_no)))

                for (s, role) in zip(result, smalls):
                    s = str(s).strip("[").strip("]")
                    htmlTag = BeautifulSoup(s, "html.parser")
                    animeStaff.append([htmlTag.text, role.text])
            
        # Create list with overall information
        final_information = [animeTitle, animeType,animeNumEpisode,releaseDate,endDate,
                         animeNumMembers, animeScore,animeUsers,animeRank,animePopularity,
                         animeDescription,animeRelated,animeCharacters,animeVoices,animeStaff]
        return final_information

#### Create a .tsv file for each anime and a .tsv file with 3 most important information for all: Title, Description, Url 

In [273]:
def create_tsvs(path_list_anime, path_page_general, path_documents_file, path_documents_ex_3):
    # Create .tsv file
    n_pages = 384

    # Collecting all Urls
    with open(path_list_anime, 'r', encoding='utf-8') as f_urls:
        urls = f_urls.readlines()
    f_urls.close

    # Create .tsv with only 3 important fields
    with open(path_documents_file, 'w', encoding='utf-8') as f:
        tsv_writer = csv.writer(f, delimiter='\t')

        # Write haeders
        tsv_writer.writerow(['animeTitle','animeDescription','Url'])

    f.close
    
    # Create .tsv unique with only 4 fields related to excercise 3
    with open(path_documents_ex_3, 'w', encoding='utf-8') as f:
        tsv_writer = csv.writer(f, delimiter='\t')

        # Write headers
        tsv_writer.writerow(['animeTitle','animeDescription','Url', 'animeScore'])

    f.close

    # Iterating through pages and then animes
    for i in range(1, n_pages):
        
        # Start and end computed with respect to the 
        start_iterate_anime = (i-1)*50+1
        end_iterate_anime = ((i-1)*50+1)+50
        for j in range(start_iterate_anime, end_iterate_anime):

            # Write for each anime a .tsv file
            with open(path_page_general + str(i) + "/anime_" + str(j)+ ".tsv", 'w', encoding='utf-8') as out_file:
                tsv_writer = csv.writer(out_file, delimiter='\t')

                # Write haeders
                tsv_writer.writerow(['animeTitle', 'animeType','animeNumEpisode','releaseDate','endDate',
                                     'animeNumMembers', 'animeScore','animeUsers','animeRank','animePopularity',
                                     'animeDescription','animeRelated','animeCharacters','animeVoices','animeStaff'])

                # Write information
                information = retrive_information(i, j) 
                tsv_writer.writerow(information)
            out_file.close

            important_information = [information[0],information[10], urls[j]]
            important_information_ex_3 = [information[0],information[10], urls[j], information[6]]

            # Append data to unique .tsv with only 3 important fields
            with open(path_documents_file, 'a', encoding='utf-8') as f:
                tsv_writer = csv.writer(f, delimiter='\t')

                # Write information 
                tsv_writer.writerow(important_information)

            f.close
            
            # Append data to unique .tsv with only 4 fields related to excercise 3
            with open(path_documents_ex_3, 'a', encoding='utf-8') as f:
                tsv_writer = csv.writer(f, delimiter='\t')

                # Write information 
                tsv_writer.writerow(important_information_ex_3)

            f.close

In [275]:
create_tsvs(path_list_anime, path_page_general, path_documents_file, path_documents_ex_3)

FileNotFoundError: [Errno 2] No such file or directory: 'C:/Users/andre/Desktop/Anime/Page5/anime_201.tsv'

In [281]:
a = df_from_4_information_file(path_documents_ex_3)
a

,animeTitle,animeDescription,Url,animeScore
0,Fullmetal Alchemist: Brotherhood,After a horrific alchemy experiment goes wrong...,https://myanimelist.net/anime/28977/Gintama°\r\n,9.16
1,Gintama°,"Gintoki, Shinpachi, and Kagura return as the f...",https://myanimelist.net/anime/38524/Shingeki_n...,9.09
2,Shingeki no Kyojin Season 3 Part 2,Seeking to restore humanity's diminishing hope...,https://myanimelist.net/anime/9253/Steins_Gate...,9.09
3,Steins;Gate,The self-proclaimed mad scientist Rintarou Oka...,https://myanimelist.net/anime/42938/Fruits_Bas...,9.09
4,Fruits Basket: The Final,"Hundreds of years ago, the Chinese Zodiac spir...",https://myanimelist.net/anime/9969/Gintama\r\n,9.06
...,...,...,...,...
195,Tenki no Ko,Tokyo is currently experiencing rain showers t...,https://myanimelist.net/anime/338/Versailles_n...,8.34
196,Versailles no Bara,"In 1755, Marie Antoinette is born in the royal...",https://myanimelist.net/anime/48653/Yoru_Ni_Ka...,8.34
197,Yoru Ni Kakeru,"Music video of the song ""Yoru Ni Kakeru"" by Yo...",https://myanimelist.net/anime/9617/K-On_Movie\r\n,8.34
198,K-On! Movie,Graduation looms for the founding members of t...,https://myanimelist.net/anime/6594/Katanagatar...,8.34


## 2. Search Engine

### Create pandas dataframe with important information

In [5]:
def df_from_3_information_file(path_documents_file):
    original_df = pd.read_csv(path_documents_file, sep='\t')
    return original_df

In [280]:
def df_from_4_information_file(path_documents_ex_3):
    original_df = pd.read_csv(path_documents_ex_3, sep='\t')
    return original_df

### Text cleaning:

In order we will do the following operations:

- Remove "[Written by MAL Rewrite]" at the end of each description

- Remove Contractions (for example "won't" become "will not" and "don't" become "do not")

- Make all characters lower case (for example "Hello" become "hello")

- Remove dashes

- Remove ordinal numbers (for example 1st, 2nd, ...)

- Remove stopwords (adding to stopwords frequent words that appear in descriptions, for example "character", "end") 

- Remove punctuation

- Stemming

In [7]:
def remove_written_mal_rewrite(text):
    return text.replace('[Written by MAL Rewrite]', '')

In [8]:
def remove_contractions(text):
    # Create patterns
    patterns = [
       (r'won\'t', 'will not'),
       (r'can\'t', 'cannot'),
       (r'i\'m', 'i am'),
       (r'(\w+)\'ll', '\g<1> will'),
       (r'(\w+)n\'t', '\g<1> not'),
       (r'(\w+)\'ve', '\g<1> have'),
       (r'(\w+)\'s', '\g<1> is'),
       (r'(\w+)\'re', '\g<1> are'),
    ]
    
    # Compile patterns with re.compile()
    patterns = [(re.compile(regex), repl) for (regex, repl) in patterns]
    
    # Substitute words
    for (pattern, repl) in patterns:
        text = re.sub(pattern, repl, text)
    return text

In [9]:
def to_lower_case(text):
    return text.lower()

In [10]:
def replace_dashes(text):
    return text.replace('—',' ')

In [11]:
def remove_ordinal_num(text):
    text = re.sub('[\d]+(st|nd|rd|th)', '', text)
    return text

In [12]:
def tokenize_text(text):
    return [w for s in sent_tokenize(text) for w in word_tokenize(s)]

In [13]:
def remove_punctuation(text):
    tokenizer = nltk.RegexpTokenizer(r"\w+")
    new_words = tokenizer.tokenize(text)
    return ' '.join(new_words)

In [14]:
def remove_stopwords(text, stop_words=default_stopwords):
        tokens = [w for w in tokenize_text(text) if w not in stop_words]
        return ' '.join(tokens)

In [15]:
def stem_text(text, stemmer=default_stemmer):
        tokens = tokenize_text(text)
        return ' '.join([stemmer.stem(t) for t in tokens])

In [16]:
def clean_text(text):
    
    # Text cleaning
    text = remove_written_mal_rewrite(text)
    text = remove_contractions(text)
    text = to_lower_case(text)
    text = replace_dashes(text)
    text = remove_ordinal_num(text)
    text = remove_punctuation(text)
    text = remove_stopwords(text)
    text = stem_text(text)
    
    return text

In [294]:
def clean_df(original_df):
    df_descriptions = original_df['animeDescription']
    df_descriptions = pd.DataFrame(df_descriptions)
    df_cleaned = df_descriptions.applymap(lambda x : clean_text(x), na_action='ignore')
    return df_cleaned

### 2.1. Conjunctive query

### 2.1.1) Create your index!

#### Create vocabulary and save it in a file .json

In [342]:
def write_vocabulary(path_vocabulary_file, df_descriptions):    
    # Create a list 
    df_words = df_descriptions.applymap(lambda x : set(tokenize_text(x)), na_action='ignore')
    words_union = set()

    for row in df_words['animeDescription']:
        words_union = words_union.union(set(row))

    words_list = list(words_union)

    # Create vocabulary
    vocabulary = { words_list[i] : i for i in range(len(words_list)) }

    # Write the file
    with open(path_vocabulary_file, 'w', encoding='utf-8') as file:
        json.dump(vocabulary, file)
    file.close

#### Create index and save it in a file .json

In [343]:
def write_index(path_index_file, df_descriptions):
    # Create the index with (id_word : [doc1, doc2, ...])
    df_words = df_descriptions.applymap(lambda x : set(tokenize_text(x)), na_action='ignore')
    index = {}
    for n_row in range(df_words['animeDescription'].shape[0]):
        for (k,v) in vocabulary.items():
            if v not in index.keys():
                index[v] = []
            if k in df_words['animeDescription'][n_row]:
                # n_row+1 becouse n_anime is n_row_in_dataset + 1
                index[v].append(n_row+1)

    # Write the file
    with open(path_index_file, 'w', encoding='utf-8') as file:
        json.dump(index, file)
    file.close

In [346]:
original_df = df_from_3_information_file(path_documents_file)
original_df = clean_df(original_df)
df_descriptions = pd.Series.to_frame(original_df.animeDescription)
write_vocabulary(path_vocabulary_file, df_descriptions)
write_index(path_index_file, df_descriptions)

In [348]:
vocabulary, index = read_vocabulary_and_index(path_vocabulary_file, path_index_file)

In [349]:
original_df.animeDescription[0]
robot = vocabulary["robot"]
robot = index[str(1484)]
robot

[1, 46, 63, 91, 125]

### 2.1.2) Execute the query

#### Load vocabulary.json and index.json as python dictionaries

In [296]:
def read_vocabulary_and_index(path_vocabulary_file, path_index_file):
    # Vocabulary
    with open(path_vocabulary_file, 'r', encoding='utf-8') as f:
        vocabulary = json.load(f)
    f.close

    # Index
    with open(path_index_file, 'r', encoding='utf-8') as f:
        index = json.load(f)
    f.close
    
    return vocabulary, index

#### Query

In [297]:
def execute_query(query, vocabulary, index):
    # Query
    query = clean_text(query).split(" ")

    # Trasform every word of the query in the corrispondant id
    id_words_query = [vocabulary[word] for word in query if word in vocabulary.keys()]

    documents_set = set()
    for id_word in id_words_query:
        if documents_set == set():
            documents_set.update(index[str(id_word)])
        else:
            documents_set = documents_set.intersection(set(index[str(id_word)]))
            if documents_set == set():
                break

    documents = list(documents_set)
    documents_in_dataframe = [x-1 for x in documents]

    # Return rows
    original_df = df_from_3_information_file(path_documents_file)
    return original_df.iloc[documents_in_dataframe]

In [302]:
vocabulary, index = read_vocabulary_and_index(path_vocabulary_file, path_index_file)

In [312]:
vocabulary

{'momo': 0,
 'sail': 1,
 'maim': 2,
 'hakas': 3,
 'avert': 4,
 'sibl': 5,
 'assort': 6,
 'meteor': 7,
 'masquerad': 8,
 'tobio': 9,
 'glimmer': 10,
 'dempsey': 11,
 'doorstep': 12,
 'reng': 13,
 'bastion': 14,
 'tip': 15,
 'foothold': 16,
 'courag': 17,
 'kouichi': 18,
 'marri': 19,
 'marley': 20,
 'ying': 21,
 'wave': 22,
 'nadeshiko': 23,
 'fund': 24,
 'daikichi': 25,
 'nap': 26,
 'york': 27,
 'gritti': 28,
 'spell': 29,
 'sunflow': 30,
 'pen': 31,
 'postal': 32,
 'boldli': 33,
 'gard': 34,
 'sibyl': 35,
 'friendli': 36,
 'south': 37,
 'crusad': 38,
 'found': 39,
 'golzin': 40,
 'aioi': 41,
 'unlucki': 42,
 'xiaofei': 43,
 'loom': 44,
 'sora': 45,
 'instead': 46,
 'premier': 47,
 'locat': 48,
 'heaviest': 49,
 'occult': 50,
 'dub': 51,
 'newborn': 52,
 'boon': 53,
 'forest': 54,
 'granzchesta': 55,
 'overpow': 56,
 'mushi': 57,
 'perfect': 58,
 'brought': 59,
 'odditi': 60,
 'imag': 61,
 'mentorship': 62,
 'enorm': 63,
 'alchem': 64,
 'peak': 65,
 'bleak': 66,
 'disdain': 67,
 'doll'

In [300]:
index

{'0': [24, 166],
 '1': [43, 120, 175],
 '2': [1, 46, 63, 91, 125],
 '3': [15],
 '4': [20],
 '5': [13],
 '6': [31],
 '7': [13, 62, 176],
 '8': [4],
 '9': [5],
 '10': [44, 54, 81, 105, 112, 120, 133, 146, 149, 157, 159, 175],
 '11': [1, 123, 143],
 '12': [2, 6, 16, 17, 25, 94, 101, 116],
 '13': [104],
 '14': [38, 57, 88, 109, 147],
 '15': [35, 145, 147],
 '16': [42, 69, 136],
 '17': [30, 71, 129],
 '18': [14],
 '19': [5, 13, 41, 58, 75, 81, 83, 91, 121, 132, 136, 168, 179],
 '20': [9, 16, 25, 33, 46, 63, 83, 116, 135, 139, 171, 181, 189, 190],
 '21': [4, 15, 65, 167],
 '22': [5],
 '23': [33, 47, 49, 53, 60, 70, 83, 93, 106, 122, 142, 148, 167, 197],
 '24': [4, 107],
 '25': [13, 39, 49, 188],
 '26': [35],
 '27': [42, 52, 114],
 '28': [16, 29, 43, 59, 112, 119, 143, 147, 161, 175, 176, 194],
 '29': [37, 90, 119],
 '30': [1, 161],
 '31': [33],
 '32': [19, 46, 50],
 '33': [34, 35, 66, 88, 119, 142, 178, 180, 182],
 '34': [41],
 '35': [36, 93],
 '36': [29, 55, 61, 118, 142, 153],
 '37': [27, 

In [350]:
rows = execute_query("robot", vocabulary, index)
rows

,animeTitle,animeDescription,Url
0,Fullmetal Alchemist: Brotherhood,After a horrific alchemy experiment goes wrong...,https://myanimelist.net/anime/28977/Gintama°\r\n
45,Made in Abyss,The Abyss—a gaping chasm stretching down into ...,https://myanimelist.net/anime/2921/Ashita_no_J...
90,Neon Genesis Evangelion: The End of Evangelion,Shinji Ikari is left emotionally comatose afte...,https://myanimelist.net/anime/38474/Yuru_Camp△...
124,Nichijou,Nichijou primarily focuses on the daily antics...,https://myanimelist.net/anime/31181/Owarimonog...
62,Tengen Toppa Gurren Lagann,Simon and Kamina were born and raised in a dee...,https://myanimelist.net/anime/34591/Natsume_Yu...


### 2.2) Conjunctive query & Ranking score

### 2.2.1) Inverted index

Calculate $tfIdf$ as:
$$
\frac{tf_{i,j}}{{|n_{i,j}|}}
$$
where $tf_{i,j}$ is the number of time the word i is present in document j and ${|n_{i,j}|}$ is the lenght of the document (number of total words)

In [379]:
original_df = df_from_3_information_file(path_documents_file)
df_cleaned = clean_df(original_df)

In [6]:
def tf_score(df_cleand, vocabulary):
    
    tf = []
    
    for num_row in range(len(df_cleaned)):
        
        doc = df_cleaned.animeDescription.loc[num_row].split()
        len_doc = len(doc)
        counter = Counter(doc)
        
        for item, count in counter.items():
            
            counter[item] /= len_doc
            
    tf.append(counter)
    tf_new = []
    
    for d in tf:
        
        dic = dict((vocabulary[key], value) for (key, value) in d.items())
        tf_new.append(dic)
        
    return tf_new

tf_new=tf_score(df_cleand, vocabulary)

NameError: name 'df_cleand' is not defined

In [5]:
def idf(de_cleaned,index):
    
    idf = {}
    num_docs = len(df_cleaned)
    
    for term_id in index:
        
        numdocs_cont_j = len(index[term_id])
        idfscore = np.log(num_docs/numdocs_cont_j)
        idf[term_id] = idfscore
        
    return idf

idf=idf(de_cleaned,index)

NameError: name 'de_cleaned' is not defined

In [4]:
def inverted_index(tf_new,idf):
    
    inverted_index = {}
    
    for term in idf: #for every term_id
    
        for i in range(len(tf_new)): #for every doc
            
            if int(term) in tf_new[i].keys(): #if term_id is the key of the document_i
                
                tupla = (i+1, idf[term]*tf_new[i][int(term)]) #note i+1
            
                if term in inverted_index:
                    inverted_index[term].append(tupla)
                    
                else:
                    inverted_index[term] = []
                    inverted_index[term].append(tupla)
                    
    return inverted_index

inverted_index=inverted_index(tf_new,idf)

In [356]:
def docs_contains_query(query,vocabulary, index):
    subset = execute_query(query, vocabulary, index)
    indexes = [i for i in subset.index]
    return indexes

In [357]:
# Get the numpy array vector for the query with frequency of words. It has the same lenght of the vocabulary and has number != 0 if the word is present in the query
def get_query_vector(query, vocabulary):
    query = clean_text(query).split(" ")
    query_vector = np.zeros(len(vocabulary.keys()))
    for word in query:
        query_vector[int(vocabulary[word])] += 1
    return query_vector

In [358]:
desc = "robot"
q_vector = get_query_vector(desc, vocabulary)

In [359]:
# Same method to get the numpy array vector of the document selected 
def get_doc_vector(n_row_doc, df_cleaned, inverted_index):
    doc = df_cleaned.animeDescription.loc[n_row_doc].split(" ")
    doc_vector = np.zeros(len(vocabulary.keys()))
    # iterating in vocabulary of terms
    for term_id, value in inverted_index.items():
        # Iterating in each tuple:
        for pair in value:
            # if the document in which the word is present is the document that i passed to this function 
            if pair[0] == n_row_doc+1:
                # Update doc_vector with the respective tfidf in the position [term_id]  
                doc_vector[int(term_id)] = pair[1]
    return doc_vector

In [360]:
doc_vector = get_doc_vector(0, df_cleaned, inverted_index)

In [361]:
def calc_score(q_vector, doc_vector):
    # Calculate cosine similarity between query vector and the selected document vector 
    score = np.dot(q_vector, doc_vector)/(math.sqrt(np.linalg.norm(q_vector))*math.sqrt(np.linalg.norm(doc_vector)))
    return score

In [362]:
score = calc_score(q_vector, doc_vector)
score

0.04996721225668058

In [363]:
def execute_similarity_query(query, k, df_cleaned, vocabulary, inverted_index, original_df):
    # Initialize the heap
    heap = []
    # Query vector
    q_vector = get_query_vector(query, vocabulary)
    
    #Manca la funzionee  docs contain query per confrontare solo con i  documenti di interesse 
    doc_list = docs_contains_query(query,vocabulary, index)
    # For all the synopsis in the dataframe
    for i in doc_list:
        doc_i_vector = get_doc_vector(i, df_cleaned, inverted_index)
        score_i = calc_score(q_vector, doc_i_vector)
        
        # Push in the heap the tuple containing (score of i-th document, i)
        if score_i  != 0:
            heapq.heappush(heap, (score_i, i))
    if heap==[]:
        return ('There are not significant similarity')
    else:
        heapq._heapify_max(heap)
        if len(heap) > k:
            heap=[heapq.heappop(heap) for i in range(k)]
        
    # List of indexes of k-documents 
    
    
        doc_rows_list = []
        score_list=[]
        for j in range(len(heap)):
            #tuple_element = heapq.heappop(heap)
            doc_rows_list.append(heap[j][1])
            score_list.append(heap[j][0])
        
        sub_df=original_df.iloc[doc_rows_list]

        sub_df['Similarity']=np.array(score_list,dtype='float') #purche l ordine con cui printiamo il sub dataset  sia lo  stesso del vettore doc_rows_list
        return  sub_df

In [369]:
execute_similarity_query("robot", 3, df_cleaned, vocabulary, inverted_index, original_df)

,animeTitle,animeDescription,Url,Similarity
124,Nichijou,Nichijou primarily focuses on the daily antics...,https://myanimelist.net/anime/31181/Owarimonog...,0.080656
0,Fullmetal Alchemist: Brotherhood,After a horrific alchemy experiment goes wrong...,https://myanimelist.net/anime/28977/Gintama°\r\n,0.049967
90,Neon Genesis Evangelion: The End of Evangelion,Shinji Ikari is left emotionally comatose afte...,https://myanimelist.net/anime/38474/Yuru_Camp△...,0.050071


### 3. Define a new score!

In [ ]:
original_df=df_from_4_information_file(path_documents_ex_3)

#### Explanation of the new score

In [371]:
def write_vocabulary_title(path_vocabulary_title_file, df_title):    
    # Create a list 
    df_words = df_title.applymap(lambda x : set(tokenize_text(x)), na_action='ignore')
    words_union = set()

    for row in df_words['animeTitle']:
        words_union = words_union.union(set(row))

    words_list = list(words_union)

    # Create vocabulary
    vocabulary = { words_list[i] : i for i in range(len(words_list)) }

    # Write the file
    with open(path_vocabulary_title_file, 'w', encoding='utf-8') as file:
        json.dump(vocabulary, file)
    file.close

In [372]:
def write_index_title(path_index_title_file, df_title):
    # Create the index with (id_word : [doc1, doc2, ...])
    df_words = df_title.applymap(lambda x : set(tokenize_text(x)), na_action='ignore')
    index = {}
    for n_row in range(df_words['animeTitle'].shape[0]):
        for (k,v) in vocabulary.items():
            if v not in index.keys():
                index[v] = []
            if k in df_words['animeTitle'][n_row]:
                # n_row+1 becouse n_anime is n_row_in_dataset + 1
                index[v].append(n_row+1)

    # Write the file
    with open(path_index_title_file, 'w', encoding='utf-8') as file:
        json.dump(index, file)
    file.close

In [373]:
def read_vocabulary_and_index_title(path_vocabulary_title_file, path_index_title_file):
    # Vocabulary
    with open(path_vocabulary_title_file, 'r', encoding='utf-8') as f:
        vocabulary = json.load(f)
    f.close

    # Index
    with open(path_index_title_file, 'r', encoding='utf-8') as f:
        index = json.load(f)
    f.close
    
    return vocabulary, index

In [382]:
def clean_df_title(original_df):
    df_title = original_df['animeTitle']
    df_title = pd.DataFrame(df_title)
    df_cleaned = df_title.applymap(lambda x : clean_text(x), na_action='ignore')
    return df_cleaned

In [384]:
df_cleaned = clean_df_title(original_df)
df_title = pd.Series.to_frame(df_cleaned.animeTitle)
write_vocabulary_title(path_vocabulary_title_file, df_title)
write_index_title(path_index_title_file, df_title)

In [385]:
vocabulary, index = read_vocabulary_and_index_title(path_vocabulary_title_file, path_index_title_file)

In [386]:
## TF
# cacola term frequency per ogni documento, ma la chiave è la parola, non l'id
tf = []
for num_row in range(len(df_cleaned)):
    doc = df_cleaned.animeTitle.loc[num_row].split()
    len_doc = len(doc)
    counter = Counter(doc)
    for item, count in counter.items():
        counter[item] /= len_doc
    tf.append(counter)
tf

[Counter({'fullmet': 0.3333333333333333,
          'alchemist': 0.3333333333333333,
          'brotherhood': 0.3333333333333333}),
 Counter({'gintama': 1.0}),
 Counter({'shingeki': 0.16666666666666666,
          'kyojin': 0.16666666666666666,
          'season': 0.16666666666666666,
          '3': 0.16666666666666666,
          'part': 0.16666666666666666,
          '2': 0.16666666666666666}),
 Counter({'stein': 0.5, 'gate': 0.5}),
 Counter({'fruit': 0.3333333333333333,
          'basket': 0.3333333333333333,
          'final': 0.3333333333333333}),
 Counter({'gintama': 1.0}),
 Counter({'hunter': 0.5, 'x': 0.25, '2011': 0.25}),
 Counter({'ginga': 0.3333333333333333,
          'eiyuu': 0.3333333333333333,
          'densetsu': 0.3333333333333333}),
 Counter({'gintama': 0.5, 'enchousen': 0.5}),
 Counter({'gintama': 0.5, 'final': 0.5}),
 Counter({'gintama': 1.0}),
 Counter({'3': 0.25, 'gatsu': 0.25, 'lion': 0.25, 'season': 0.25}),
 Counter({'koe': 0.5, 'katachi': 0.5}),
 Counter({'violet'

In [389]:
print(len(index), len(vocabulary))

452 420


In [387]:
## IDF
idf = {}
import numpy as np
numdocs_cont_j = {}
num_docs = len(df_cleaned)
for term_id in index:
    numdocs_cont_j = len(index[term_id])
    idfscore = np.log(num_docs/numdocs_cont_j)
    idf[term_id] = idfscore
idf

ZeroDivisionError: division by zero

In [354]:
## per mappare le chiavi di tf in numeri(id)
tf_new = []
for d in tf:
    dic = dict((vocabulary[key], value) for (key, value) in d.items())
    tf_new.append(dic)
tf_new

[{2000: 0.009009009009009009,
  1301: 0.009009009009009009,
  275: 0.009009009009009009,
  801: 0.009009009009009009,
  3134: 0.009009009009009009,
  3116: 0.018018018018018018,
  267: 0.009009009009009009,
  2201: 0.02702702702702703,
  526: 0.036036036036036036,
  1928: 0.02702702702702703,
  975: 0.009009009009009009,
  3543: 0.009009009009009009,
  3548: 0.009009009009009009,
  1713: 0.009009009009009009,
  3920: 0.009009009009009009,
  64: 0.009009009009009009,
  4198: 0.009009009009009009,
  1448: 0.009009009009009009,
  360: 0.009009009009009009,
  580: 0.009009009009009009,
  2816: 0.018018018018018018,
  1808: 0.009009009009009009,
  1101: 0.009009009009009009,
  3838: 0.009009009009009009,
  2530: 0.009009009009009009,
  3691: 0.009009009009009009,
  1391: 0.009009009009009009,
  3205: 0.009009009009009009,
  46: 0.009009009009009009,
  3799: 0.009009009009009009,
  1798: 0.009009009009009009,
  2959: 0.009009009009009009,
  1698: 0.009009009009009009,
  4258: 0.0180180180180

In [355]:
inverted_index = {}
for term in idf: #for every term_id
    
    for i in range(len(tf_new)): #for every doc
        if int(term) in tf_new[i].keys(): #if term_id is the key of the document_i
            tupla = (i+1, idf[term]*tf_new[i][int(term)]) #note i+1
            
            if term in inverted_index:
                inverted_index[term].append(tupla)
            else:
                inverted_index[term] = []
                inverted_index[term].append(tupla)
inverted_index

{'0': [(12, 0.05548397814443484), (160, 0.04514872731360874)],
 '1': [(71, 0.041117590946322245), (90, 0.03837641821656743)],
 '2': [(56, 0.05953165580391052)],
 '3': [(125, 0.08278620885231307)],
 '4': [(85, 0.06772309097041311), (118, 0.04699153251008257)],
 '5': [(82, 0.03371690285884598),
  (84, 0.046138919701578705),
  (88, 0.037303807418297676),
  (109, 0.037704923627096584),
  (113, 0.04276290118682905),
  (190, 0.05081967967130408)],
 '6': [(143, 0.06383514899455466)],
 '7': [(164, 0.06792714572497482)],
 '8': [(133, 0.06461362642131752)],
 '9': [(127, 0.044280482557577806), (186, 0.04344500175460464)],
 '10': [(14, 0.07195578415606393), (189, 0.036841361487904734)],
 '11': [(79, 0.06383514899455466)],
 '12': [(34, 0.04688776430573483)],
 '13': [(152, 0.08830528944246727)],
 '14': [(35, 0.05351835723785896)],
 '15': [(89, 0.046516870565536286), (120, 0.05060626578008892)],
 '16': [(34, 0.04688776430573483)],
 '17': [(30, 0.0559960677050657),
  (148, 0.03499754231566606),
  (194

In [ ]:
def execute_new_score_query(query, score = False, title = False, df_from_4_information_file):
    